In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Carregar dados limpos
df = pd.read_csv(r'C:\Users\João Pedro\Documents\UFG\AMS\AS2\data\train_cleaned.csv')

def feature_engineering_trees(df):
    df_feat = df.copy()
    
    # 1. CABINE (Deck/Num/Side) - Crucial para árvores
    if 'Cabin' in df_feat.columns:
        df_feat[['Deck', 'Num', 'Side']] = df_feat['Cabin'].str.split('/', expand=True)
        # Converter Num para int (árvores adoram sequências numéricas)
        df_feat['Num'] = pd.to_numeric(df_feat['Num'], errors='coerce').fillna(0)
    
    # 2. GASTOS (Total e Log)
    cols_gastos = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    df_feat['TotalSpend'] = df_feat[cols_gastos].sum(axis=1)
    
    # Log Transform ajuda o XGBoost a achar melhores pontos de corte em dados financeiros
    # (Para o CatBoost é menos crítico, mas ajuda)
    for col in cols_gastos + ['TotalSpend', 'Num']:
        if col in df_feat.columns:
            df_feat[col] = np.log1p(df_feat[col])
            
    # 3. FORMATAR BOOLEANOS (True/False -> 1/0)
    # Árvores preferem inteiros
    bool_cols = ['CryoSleep', 'VIP']
    for col in bool_cols:
        if col in df_feat.columns:
            df_feat[col] = df_feat[col].astype(int)

    # 4. LIMPEZA
    # Removemos colunas de alta cardinalidade que não ajudam árvores
    # Group e Surname podem ser removidos pois já temos GroupSize e FamilySize
    cols_drop = ['PassengerId', 'Name', 'Cabin', 'Group', 'Surname']
    df_feat = df_feat.drop(columns=[c for c in cols_drop if c in df_feat.columns])
    
    return df_feat

# Aplicar engenharia
df_processed = feature_engineering_trees(df)

# Separar X e y
X = df_processed.drop('Transported', axis=1)
y = df_processed['Transported'].astype(int)

# Split de validação (essencial para Early Stopping)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print("Features disponíveis:", list(X.columns))

Features disponíveis: ['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'FamilySize', 'GroupSize', 'Deck', 'Num', 'Side', 'TotalSpend']


In [2]:
from catboost import CatBoostClassifier, Pool

# 1. Identificar colunas categóricas (Texto)
# O CatBoost precisa dos índices ou nomes dessas colunas
cat_features = ['HomePlanet', 'Destination', 'Deck', 'Side']

# Certifique-se de que são strings (evita erros se tiver nulos misturados)
X_train[cat_features] = X_train[cat_features].astype(str)
X_val[cat_features] = X_val[cat_features].astype(str)

# 2. Criar Pools (Otimiza o carregamento de dados)
train_pool = Pool(data=X_train, label=y_train, cat_features=cat_features)
val_pool = Pool(data=X_val, label=y_val, cat_features=cat_features)

# 3. Definir e Treinar Modelo
model_cat = CatBoostClassifier(
    iterations=1000,        # Número máximo de árvores
    learning_rate=0.03,     # Velocidade de aprendizado (menor = mais preciso, mas demora mais)
    depth=6,                # Profundidade da árvore
    eval_metric='Accuracy', # Métrica para monitorar
    verbose=100,            # Imprimir a cada 100 iterações
    early_stopping_rounds=50 # Para se parar de melhorar
)

print("\n--- Treinando CatBoost ---")
model_cat.fit(train_pool, eval_set=val_pool)

print(f"Melhor Acurácia Validação: {model_cat.get_best_score()['validation']['Accuracy']:.4f}")

# Salvar
model_cat.save_model('modelo_catboost.cbm')


--- Treinando CatBoost ---
0:	learn: 0.7621513	test: 0.7538815	best: 0.7538815 (0)	total: 169ms	remaining: 2m 48s
100:	learn: 0.8126258	test: 0.7970098	best: 0.7970098 (100)	total: 3.88s	remaining: 34.6s
200:	learn: 0.8283003	test: 0.7987349	best: 0.8004600 (185)	total: 7.71s	remaining: 30.7s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.8004600345
bestIteration = 185

Shrink model to first 186 iterations.
Melhor Acurácia Validação: 0.8005


In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool

# --- 1. FUNÇÃO DE PREPARAÇÃO (Feature Engineering) ---
def preparar_dados_catboost(df, eh_treino=True):
    df_proc = df.copy()
    
    # A) Tratamento de Nulos (Essencial para o Teste)
    # Preencher numéricos com 0
    num_cols = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    for col in num_cols:
        if col in df_proc.columns:
            df_proc[col] = df_proc[col].fillna(0)
            
    # Preencher categóricos com "Unknown" (CatBoost adora isso)
    cat_cols_raw = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Cabin']
    for col in cat_cols_raw:
        if col in df_proc.columns:
            df_proc[col] = df_proc[col].fillna('Unknown')

    # B) Engenharia de Cabine
    if 'Cabin' in df_proc.columns:
        # Cria colunas temporárias. Se for 'Unknown', vira 'Unknown'
        cabine_split = df_proc['Cabin'].str.split('/', expand=True)
        # Se o split falhar (por ser Unknown), preenchemos
        if len(cabine_split.columns) == 3:
            df_proc['Deck'] = cabine_split[0].fillna('Unknown')
            df_proc['Num'] = pd.to_numeric(cabine_split[1], errors='coerce').fillna(0)
            df_proc['Side'] = cabine_split[2].fillna('Unknown')
        else:
            # Fallback seguro caso tudo seja nulo
            df_proc['Deck'] = 'Unknown'
            df_proc['Num'] = 0
            df_proc['Side'] = 'Unknown'

    # C) Engenharia de Gastos
    cols_gastos = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    # Garante que as colunas existem
    cols_existentes = [c for c in cols_gastos if c in df_proc.columns]
    df_proc['TotalSpend'] = df_proc[cols_existentes].sum(axis=1)
    
    # Log Transform (Ajuda muito árvores com valores financeiros)
    cols_to_log = cols_existentes + ['TotalSpend', 'Num']
    for col in cols_to_log:
        df_proc[col] = np.log1p(df_proc[col])

    # D) Seleção e Tipagem
    # CatBoost precisa que categóricas sejam Strings e numéricas sejam int/float
    # Colunas que vamos usar no model_cato:
    features_cat = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side']
    features_num = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'TotalSpend', 'Num']
    
    # Garantir que as colunas categóricas existam e sejam strings
    for col in features_cat:
        if col not in df_proc.columns:
            df_proc[col] = 'Unknown'
        df_proc[col] = df_proc[col].astype(str)
        
    features_finais = features_cat + features_num
    
    if eh_treino:
        return df_proc[features_finais], df_proc['Transported'].astype(int)
    else:
        return df_proc[features_finais]

# --- 2. CARREGAR DADOS ---
print("Carregando dados...")
# Substitua pelos seus caminhos se necessário, ou mantenha se estiver na mesma pasta
df_train = pd.read_csv(r'C:\Users\João Pedro\Documents\UFG\AMS\AS2\data\train_cleaned.csv') 
df_test = pd.read_csv(r'C:\Users\João Pedro\Documents\UFG\AMS\AS2\data\test_cleaned.csv')

# Guardar IDs para submissão
passenger_ids = df_test['PassengerId'].copy()

# --- 3. PROCESSAR ---
print("Processando features...")
X_train, y_train = preparar_dados_catboost(df_train, eh_treino=True)
X_test_cat = preparar_dados_catboost(df_test, eh_treino=False)

# Identificar índices das colunas categóricas para o CatBoost
cat_features_indices = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side']
print(f"Features Categóricas: {cat_features_indices}")

# --- 4. TREINAR O CATBOOST ---
print("Treinando model_cato CatBoost...")
model_cat = CatBoostClassifier(
    iterations=1500,        # Número de árvores
    learning_rate=0.02,     # Taxa de aprendizado (menor = mais preciso, mas demora mais)
    depth=6,                # Profundidade da árvore
    eval_metric='Accuracy', 
    verbose=100,            # Imprimir progresso a cada 100
    random_seed=42
)

# Treinar (Não precisa de validação aqui, vamos usar todos os dados para o model_cato final)
model_cat.fit(X_train, y_train, cat_features=cat_features_indices)

# # --- 5. PREDIÇÃO E SUBMISSÃO ---
# print("Realizando predições...")
# y_pred = model_cat.predict(X_test)

# # O CatBoost retorna 0 e 1 (porque convertemos o y_train para int). 
# # Precisamos converter de volta para True/False (Booleano) para o Kaggle aceitar.
# y_pred_bool = y_pred.astype(bool)

# # Criar DataFrame
# submission = pd.DataFrame({
#     'PassengerId': passenger_ids,
#     'Transported': y_pred_bool
# })

# # Salvar
# submission.to_csv('submission_catboost.csv', index=False)
# print("\nSucesso! Arquivo 'submission_catboost.csv' gerado.")
# print(submission.head())

Carregando dados...
Processando features...
Features Categóricas: ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side']
Treinando modelo CatBoost...
0:	learn: 0.7665938	total: 51.1ms	remaining: 1m 16s
100:	learn: 0.8015645	total: 3.43s	remaining: 47.5s
200:	learn: 0.8107673	total: 6.87s	remaining: 44.4s
300:	learn: 0.8193949	total: 10.3s	remaining: 41s
400:	learn: 0.8266421	total: 14.1s	remaining: 38.7s
500:	learn: 0.8341194	total: 18s	remaining: 35.9s
600:	learn: 0.8389509	total: 21.7s	remaining: 32.4s
700:	learn: 0.8434372	total: 25.7s	remaining: 29.3s
800:	learn: 0.8480387	total: 29.4s	remaining: 25.7s
900:	learn: 0.8536754	total: 33.1s	remaining: 22s
1000:	learn: 0.8574715	total: 36.8s	remaining: 18.4s
1100:	learn: 0.8625331	total: 40.8s	remaining: 14.8s
1200:	learn: 0.8673645	total: 44.8s	remaining: 11.1s
1300:	learn: 0.8697803	total: 49.1s	remaining: 7.5s
1400:	learn: 0.8741516	total: 53.5s	remaining: 3.78s
1499:	learn: 0.8779478	total: 57.8s	remaining: 0us
Realizando

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

# --- 1. DEFINIR FUNÇÕES DE LIMPEZA (A mesma lógica usada antes) ---
def feature_engineering(df):
    df_feat = df.copy()
    
    # Tratamento básico de nulos para garantir que as funções funcionem
    num_cols = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    for c in num_cols:
        if c in df_feat.columns: df_feat[c] = df_feat[c].fillna(0)
    
    cat_cols = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Cabin']
    for c in cat_cols:
        if c in df_feat.columns: df_feat[c] = df_feat[c].fillna(df_feat[c].mode()[0] if not df_feat[c].mode().empty else 'Unknown')

    # Engenharia
    if 'Cabin' in df_feat.columns:
        df_feat[['Deck', 'Num', 'Side']] = df_feat['Cabin'].str.split('/', expand=True)
        df_feat['Num'] = pd.to_numeric(df_feat['Num'], errors='coerce').fillna(0)
    
    # Recriar features derivadas
    cols_gastos = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    df_feat['TotalSpend'] = df_feat[cols_gastos].sum(axis=1)
    
    # Log Transform
    cols_to_log = cols_gastos + ['TotalSpend', 'Num']
    for col in cols_to_log:
        if col in df_feat.columns:
            df_feat[col] = np.log1p(df_feat[col])
            
    return df_feat

# --- 2. RECONSTRUIR O PREPROCESSOR (Usando o TREINO) ---
print("1. Recarregando dados de Treino para ajustar as colunas...")
df_train = pd.read_csv(r'C:\Users\João Pedro\Documents\UFG\AMS\AS2\data\train_cleaned.csv') # Seu arquivo de treino original
df_train_proc = feature_engineering(df_train)

# Definir colunas para o modelo
cols_drop = ['PassengerId', 'Name', 'Cabin', 'Group', 'Surname', 'Transported']
features = [c for c in df_train_proc.columns if c not in cols_drop]

# Separar tipos
X_train_raw = df_train_proc[features]
num_cols = [c for c in X_train_raw.columns if X_train_raw[c].dtype in ['int64', 'float64', 'int32', 'float32']]
cat_cols = [c for c in X_train_raw.columns if X_train_raw[c].dtype == 'object' or X_train_raw[c].dtype == 'bool']

# CRIAR E TREINAR O PROCESSADOR
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False), cat_cols)
    ],
    verbose_feature_names_out=False
)
preprocessor.fit(X_train_raw) # O Preprocessor aprende aqui (26 colunas)

# --- 3. PROCESSAR O TESTE (Usando o cérebro do Treino) ---
print("2. Processando dados de Teste...")
df_test = pd.read_csv(r'C:\Users\João Pedro\Documents\UFG\AMS\AS2\data\test_cleaned.csv')
ids_kaggle = df_test['PassengerId'].copy()

# Engenharia no Teste
df_test_proc = feature_engineering(df_test)

# Garantir que o teste tenha apenas as colunas esperadas
X_test_raw = df_test_proc[features] # Seleciona as mesmas colunas do treino

# TRANSFORMAR (Atenção: Apenas transform, usando o preprocessor ajustado acima)
X_test_final = preprocessor.transform(X_test_raw)

print(f"Shape esperado pelo modelo: (N, 26)")
print(f"Shape gerado agora: {X_test_final.shape}")

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

# Supondo que você já tem as previsões de probabilidade (não o booleano final!)
# Se você só tem os CSVs finais com True/False, não funciona tão bem.
# O ideal é pegar o .predict() bruto antes do .astype(bool).

# 1. Obter probabilidades da Rede Neural
# (Certifique-se de usar o mesmo X_test tratado para NN)

model_path = r'C:\Users\João Pedro\Documents\UFG\AMS\AS2\model\melhor_modelo_spaceship.keras'
model_nn = tf.keras.models.load_model(model_path)
pred_prob_nn = model_nn.predict(X_test_final).flatten() 

# 2. Obter probabilidades do CatBoost
# (Certifique-se de usar o X_test tratado para CatBoost)
# O CatBoost retorna [Prob_Classe0, Prob_Classe1], queremos a coluna 1
pred_prob_cat = model_cat.predict_proba(X_test_cat)[:, 1]

# 3. MISTURAR (Ensemble)
# Vamos confiar 60% no CatBoost e 40% na Rede Neural
peso_cat = 0.6
peso_nn = 0.4

pred_final_prob = (pred_prob_cat * peso_cat) + (pred_prob_nn * peso_nn)

# 4. Decisão Final
pred_final_bool = (pred_final_prob >= 0.5).astype(bool)

# 5. Gerar Submissão Híbrida
submission = pd.DataFrame({
    'PassengerId': passenger_ids,
    'Transported': pred_final_bool
})

submission.to_csv('submission_ensemble.csv', index=False)
print("Ensemble gerado com sucesso!")